In [ ]:
import pandas as pd
from graphdatascience import GraphDataScience
from dotenv import load_dotenv
import os

: 

In [ ]:
load_dotenv()

In [ ]:
gds = GraphDataScience(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASS"))
)
gds.set_database("neo4j")